# Guest House - Easy

Data for this assessment is available:

- [guesthouse data in MySQL format](http://sqlzoo.net/guesthouse.sql)
- [guesthouse data in Microsoft SQL Server format](http://sqlzoo.net/guesthouse-ms.sql)

Background

- Guests stay at a small hotel.
- Each booking is recorded in the table booking, the date of the first night of the booking is stored here (we do not record the date the booking was made)
- At the time of booking the room to be used is decided
- There are several room types (single, double..)
- The amount charged depends on the room type and the number of people staying and the number of nights
- Guests may be charged extras (for breakfast or using the minibar)
- **Database Description** | [Easy Problems](https://sqlzoo.net/wiki/Guest_House_Assessment_Easy) | [Medium Problems](https://sqlzoo.net/wiki/Guest_House_Assessment_Medium) | [Hard Problems](https://sqlzoo.net/wiki/Guest_House_Assessment_Hard)
- [Guest House Assessment Sample Queries](https://sqlzoo.net/wiki/Guest_House_Assessment_Sample_Queries)

![rel](https://sqlzoo.net/w/images/8/83/Hotel.png)

## Table booking
The table booking contains an entry for every booking made at the hotel. A booking is made by one guest - even though more than one person may be staying we do not record the details of other guests in the same room. In normal operation the table includes both past and future bookings.

```
+----------+------------+-------+--------+---------+-------------------+------+------------+
|booking_id|booking_date|room_no|guest_id|occupants|room_type_requested|nights|arrival_time|
+----------+------------+-------+--------+---------+-------------------+------+------------+
|     5001 | 2016-11-03 |   101 |   1027 |       1 | single            |    7 | 13:00      |
|     5002 | 2016-11-03 |   102 |   1179 |       1 | double            |    2 | 18:00      |
|     5003 | 2016-11-03 |   103 |   1106 |       2 | double            |    2 | 21:00      |
|     5004 | 2016-11-03 |   104 |   1238 |       1 | double            |    3 | 22:00      |
+----------+------------+-------+--------+---------+-------------------+------+------------+
```

## Table room
Rooms are either single, double, twin or family.

```
+-----+-----------+---------------+
| id  | room_type | max_occupancy |
+-----+-----------+---------------+
| 101 | single    |             1 |
| 102 | double    |             2 |
| 103 | double    |             2 |
| 104 | double    |             2 |
| 105 | family    |             3 |
+-----+-----------+---------------+
```

## Table rate
Rooms are charged per night, the amount charged depends on the "room type requested" value of the booking and the number of people staying:

```
+-----------+-----------+--------+
| room_type | occupancy | amount |
+-----------+-----------+--------+
| double    |         1 |  56.00 |
| double    |         2 |  72.00 |
| family    |         1 |  56.00 |
| family    |         2 |  72.00 |
| family    |         3 |  84.00 |
| single    |         1 |  48.00 |
| twin      |         1 |  50.00 |
| twin      |         2 |  72.00 |
+-----------+-----------+--------+
```

You can see that a double room with one person staying costs £56 while a double room with 2 people staying costs £72 per night

Note that the actual room assigned to the booking might not match the room required (a customer may ask for a single room but we actually assign her a double). In this case we charge at the "requirement rate".

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   0.8.5
✔ tidyr   1.0.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Password? ····


## 1.
Guest 1183. Give the booking_date and the number of nights for guest 1183.

```
+--------------+--------+
| booking_date | nights |
+--------------+--------+
| 2016-11-27   |      5 |
+--------------+--------+
```

In [2]:
rate <- dbReadTable(con, 'rate')
extra <- dbReadTable(con, 'extra')
booking <- dbReadTable(con, 'booking')
room <- dbReadTable(con, 'room')
room_type <- dbReadTable(con, 'room_type')
guest <- dbReadTable(con, 'guest')

In [3]:
booking %>%
    filter(booking_date==as.Date('2016-11-27') &
           guest_id==1183) %>%
    select(booking_date, nights)

booking_date,nights
<date>,<int>
2016-11-27,5


## 2.
When do they get here? List the arrival time and the first and last names for all guests due to arrive on 2016-11-05, order the output by time of arrival.

```
+--------------+------------+-----------+
| arrival_time | first_name | last_name |
+--------------+------------+-----------+
| 14:00        | Lisa       | Nandy     |
| 15:00        | Jack       | Dromey    |
| 16:00        | Mr Andrew  | Tyrie     |
| 21:00        | James      | Heappey   |
| 22:00        | Justin     | Tomlinson |
+--------------+------------+-----------+
```

In [4]:
booking %>%
    filter(as.Date(booking_date)==as.Date('2016-11-05')) %>%
    inner_join(guest, by=c(guest_id="id")) %>%
    select(arrival_time, first_name, last_name) %>%
    arrange(arrival_time)

arrival_time,first_name,last_name
<chr>,<chr>,<chr>
14:00,Lisa,Nandy
15:00,Jack,Dromey
16:00,Mr Andrew,Tyrie
21:00,James,Heappey
22:00,Justin,Tomlinson


## 3.
Look up daily rates. Give the daily rate that should be paid for bookings with ids 5152, 5165, 5154 and 5295. Include booking id, room type, number of occupants and the amount.

```
+------------+---------------------+-----------+--------+
| booking_id | room_type_requested | occupants | amount |
+------------+---------------------+-----------+--------+
|       5152 | double              |         2 |  72.00 |
|       5154 | double              |         1 |  56.00 |
|       5295 | family              |         3 |  84.00 |
+------------+---------------------+-----------+--------+
```

In [5]:
booking %>%
    filter(booking_id %in% c(5152, 6165, 5154, 5295)) %>%
    inner_join(room, by=c(room_no="id")) %>%
    inner_join(rate, by=c(room_type="room_type", occupants="occupancy")) %>%
    select(booking_id, room_type_requested, occupants, amount)

booking_id,room_type_requested,occupants,amount
<int>,<chr>,<int>,<dbl>
5152,double,2,72
5154,double,1,56
5295,family,3,84


## 4.
Who’s in 101? Find who is staying in room 101 on 2016-12-03, include first name, last name and address.

```
+------------+-----------+-------------+
| first_name | last_name | address     |
+------------+-----------+-------------+
| Graham     | Evans     | Weaver Vale |
+------------+-----------+-------------+
```

In [6]:
booking %>%
    filter(room_no==101 &
           booking_date <= as.Date('2016-12-03') + nights &
           booking_date >= as.Date('2016-12-03')) %>%
    inner_join(guest, by=c(guest_id="id")) %>%
    select(first_name, last_name, address)

first_name,last_name,address
<chr>,<chr>,<chr>
Graham,Evans,Weaver Vale


## 5.
How many bookings, how many nights? For guests 1185 and 1270 show the number of bookings made and the total number of nights. Your output should include the guest id and the total number of bookings and the total number of nights.

```
+----------+---------------+-------------+
| guest_id | COUNT(nights) | SUM(nights) |
+----------+---------------+-------------+
|     1185 |             3 |           8 |
|     1270 |             2 |           3 |
+----------+---------------+-------------+
```

In [7]:
booking %>%
    filter(guest_id %in% c(1185, 1270)) %>%
    group_by(guest_id) %>%
    summarise(n_booking=n(), n_nights=sum(nights))

guest_id,n_booking,n_nights
<int>,<int>,<int>
1185,3,8
1270,2,3


In [8]:
dbDisconnect(con)